In [ ]:
!pip install pyspark

!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

**Cargar el archivo CSV de Walmart Stock, hacer que Spark infiera los tipos de datos.**



In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

csv_path = '/content/walmart_stock.csv'

if os.path.exists(csv_path):
    print("El archivo CSV existe en Google Drive.")
else:
    print("El archivo CSV no se encuentra en la ruta especificada.")

df = spark.read.option("header", "true").option("inferSchema", "true").csv(csv_path)





El archivo CSV existe en Google Drive.


**Cuáles son los nombres de las columnas**

In [ ]:
column_names = df.columns
print(column_names)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


**¿Qué aspecto tiene el esquema?**

In [ ]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



**Imprime las 5 primeras columnas.**

In [ ]:
df.select(first_5_columns).show(5)

+----------+------------------+---------+---------+------------------+
|      Date|              Open|     High|      Low|             Close|
+----------+------------------+---------+---------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18|
+----------+------------------+---------+---------+------------------+
only showing top 5 rows



**Utiliza describe() para conocer el DataFrame.**

In [ ]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

**Hay demasiados decimales para la media y el stddev en describe(). Formatea los números para que sólo se muestren hasta dos decimales. Presta atención a los tipos de datos que devuelve .describe()**

In [ ]:
from pyspark.sql.functions import format_number
describe_df = df.describe()
describe_df.printSchema()



root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [ ]:
formatted_df = describe_df.select(
    describe_df["summary"],
    format_number(describe_df["Open"].cast("double"), 2).alias("Open"),
    format_number(describe_df["High"].cast("double"), 2).alias("High"),
    format_number(describe_df["Low"].cast("double"), 2).alias("Low"),
    format_number(describe_df["Close"].cast("double"), 2).alias("Close")
)

formatted_df.show()

+-------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|
+-------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39|
| stddev|    6.77|    6.77|    6.74|    6.76|
|    min|   56.39|   57.06|   56.30|   56.42|
|    max|   90.80|   90.97|   89.25|   90.47|
+-------+--------+--------+--------+--------+



**Crea un nuevo dataframe con una columna llamada HV Ratio que es la relación entre el precio máximo y el volumen de las acciones negociadas durante un día**

In [ ]:
from pyspark.sql.functions import col
df_with_HV = df.withColumn("HV Ratio", col("High") / col("Volume"))
df_with_HV.select("HV Ratio").show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



**¿Qué día hubo el pico máximo en el precio?**

In [ ]:
max_price_day = df.orderBy(col("High").desc()).first()
print(f"El dia del precio maximo  {max_price_day['Date']} el precio fue {max_price_day['High']}")

El dia del precio maximo  2015-01-13 el precio fue 90.970001


**¿Cuál es la media de la columna Close?**

In [ ]:
from pyspark.sql.functions import avg
mean_close = df.agg(avg("Close")).collect()[0][0]
print(f"La media es: {mean_close}")

La media es: 72.38844998012726


**¿Cuál es el máximo y el mínimo de la columna Volumen?**

In [ ]:
from pyspark.sql.functions import max, min
volume_stats = df.agg(
    max("Volume").alias("Max Volume"),
    min("Volume").alias("Min Volume")
).collect()[0]

print(f"El maximo es: {volume_stats['Max Volume']}")
print(f"El minimo es: {volume_stats['Min Volume']}")

El maximo es: 80898100
El minimo es: 2094900


**¿Cuántos días estuvo el cierre por debajo de los 60 dólares?**

In [ ]:
days_below_60 = df.filter(col("Close") < 60).count()
print(f" {days_below_60}")

 81


**¿Qué porcentaje de veces el Máximo fue superior a 80 dólares?**
**En otras palabras, Número de días de máximos>80 Días totales en el conjunto de datos **

In [ ]:
days_max_above_80 = df.filter(col("High") > 80).count()
total_days = df.count()
porcentage = (days_max_above_80 / total_days) * 100
print(f"{porcentage:.2f}%")

9.14%


**¿Cuál es la correlación de Pearson entre High y Volume?**

In [ ]:
correlation = df.corr("High", "Volume")
print(f"La correlacion es: {correlation:.2f}")

La correlacion es: -0.34


**¿Cuál es el valor máximo de High por año?**

In [ ]:
from pyspark.sql.functions import year, col, max
df = df.withColumn("Date", col("Date").cast("date"))
df_with_year = df.withColumn("Year", year(col("Date")))
max_high_by_year = df_with_year.groupBy("Year").agg(max("High").alias("Max High"))
max_high_by_year.show()

+----+---------+
|Year| Max High|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



**¿Cuál es el cierre medio de cada mes del calendario?**
**En otras palabras, a lo largo de todos los años, ¿cuál es el precio medio de cierre para enero, febrero, marzo, etc.**? **Su resultado tendrá un valor para cada uno de estos meses. **

In [ ]:
from pyspark.sql.functions import month, avg, col

df = df.withColumn("Date", col("Date").cast("date"))
df_with_month = df.withColumn("Month", month(col("Date")))
avg_close_by_month = df_with_month.groupBy("Month").agg(avg("Close").alias("Avg Close"))
avg_close_by_month.orderBy("Month").show()

+-----+-----------------+
|Month|        Avg Close|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

